# workerノードの削除

---

CoursewareHub環境を構成しているworkerノードから一部のノードを削除します。

## パラメータの指定

workerノードを削除するのに必要となるパラメータを入力します。

### VCCアクセストークンの入力

VCノードを削除するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass

vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

#### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path("ansible.cfg")
if cfg_ansible.exists():
    os.environ["ANSIBLE_CONFIG"] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f"{ugroup_name}_manager"

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f"{ugroup_name}_worker"

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### 削除するノードの指定

workerノードから削除するノードを指定します。

削除するノードの指定は二通りの方法で行うことができます。

* 削除するworkerノード数を指定する
* 削除するノードのIPアドレスを指定する

ノードの削除を行う前に、現在のworkerノードの状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(ugroup_name)
unit_worker = ugroup.get_unit("worker")
unit_worker.df_nodes()

#### 削除するworkerノード数を指定する

> 削除するノードをIPアドレスで指定する場合はこの節をスキップしてください。

削除するノード数を指定してください。

In [ ]:
# (例)
# delete_nodes = 2

delete_nodes = 

#### 削除するノードのIPアドレスを指定する

> 削除するノード数を指定した場合はこの節をスキップしてください。ノード数とIPアドレスの両方を指定した場合はIPアドレスの指定が優先されます。

削除するノードのIPアドレスのリストを指定してください。

In [ ]:
# (例)
# delete_ipaddresses = [
#     '172.30.2.101',
#     '172.30.2.102',
#     '172.30.2.103',
# ]

delete_ipaddresses = [
    
]

#### チェック

指定されたパラメータをチェックします。

In [ ]:
worker_addrs = unit_worker.find_ip_addresses()
if "delete_ipaddresses" in vars() and len(delete_ipaddresses) > 0:
    if not set(delete_ipaddresses) <= set(worker_addrs):
        raise RuntimeError("workerノードでないIPアドレスが指定されています")
elif "delete_nodes" in vars():
    if delete_nodes > len(worker_addrs):
        raise RuntimeError("現在のworkerノード数よりも大きなノード数が指定されています")

## ノードの削除

### VCノードの削除

VCノードを削除します。

In [ ]:
params = {}
if "delete_ipaddresses" in vars() and len(delete_ipaddresses) > 0:
    params["ip_addresses"] = delete_ipaddresses
elif "delete_nodes" in vars():
    params["num_delete_nodes"] = delete_nodes
unit_worker.delete_nodes(**params)

削除後のworkerノードの状態を確認します。

In [ ]:
unit_worker.df_nodes()

### Ansibleの設定を更新する

Ansibleのインベントリから削除したノードの情報を削除します。

In [ ]:
from pathlib import Path
import yaml

inventory_path = Path("inventory.yml")
with inventory_path.open() as f:
    inventory = yaml.safe_load(f)
inventory["all"]["children"][ugroup_name]["children"][
    f"{ugroup_name}_{unit_worker.name}"
]["hosts"] = dict([(x, dict(servicenet_ip=x)) for x in unit_worker.find_ip_addresses()])
bak_inventory_path = Path(inventory_path.parent, inventory_path.name + ".bak")
inventory_path.rename(bak_inventory_path)
with inventory_path.open(mode="w") as f:
    yaml.safe_dump(inventory, f)

変更差分を確認します。

In [ ]:
! ! diff -u {bak_inventory_path} {inventory_path}

group_varsに記録されているworkerノードのIPアドレスとノード数を更新します。

In [ ]:
%run scripts/group.py
worker_ipaddresses = unit_worker.find_ip_addresses()
update_group_vars(
    ugroup_name,
    worker_nodes=len(worker_ipaddresses),
    worker_ipaddresses=worker_ipaddresses,
)

### NFSサーバ

NFSサーバが属しているansibleのグループ名を指定します。

In [ ]:
if 'nfs_target' in gvars:
    nfs_group = gvars['nfs_target']
elif 'nfs_ipaddress' in gvars:
    nfs_group = f'{ugroup_name}_nfs'
else:
    nfs_group = f'{ugroup_name}_manager'

指定されたグループのノードを操作できることを確認します。

In [ ]:
!ansible {nfs_group} -m ping

In [ ]:
!ansible {nfs_group} -b -a 'whoami'

### /etc/exportsを更新する

NFSサーバの`/etc/exports`から削除したノードを削除して設定を更新します。

`/etc/exports`の記述を更新します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

exports_opts = 'rw,fsid=0,no_root_squash,no_subtree_check,sync,crossmnt'
with TemporaryDirectory() as workdir:
        exports = Path(workdir) / f'{ugroup_name}.exports'
        with exports.open(mode='w') as f:
            if 'nfs_ipaddress' in gvars:
                addr = gvars["manager_ipaddress"]
                f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
            for addr in unit_worker.find_ip_addresses():
                f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
        !cat {exports}
        !ansible {nfs_group} -bDv -m copy -a \
                'src={exports} dest=/etc/exports.d/ backup=yes'

設定ファイルを読み込ませてエクスポート設定を更新します。

In [ ]:
!ansible {nfs_group} -b -a 'exportfs -r -v'

### Docker Swarmを更新する

削除したノードをDocker Swarmから削除します。

Docker Swarmを構成しているworkerノードの一覧を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker node ls -f role=worker'

削除したノードは上記の一覧表示で`STATUS`が`Down`と表示されています。このノードをDocker Swarmから削除します。

In [ ]:
out = !ansible {ugroup_name}_manager -a \
    'docker node ls -f role=worker --format "{{% raw %}}{{{{.ID}}}} {{{{.Status}}}}{{% endraw %}}"' 2> /dev/null

for node_id, status in [x.split(' ') for x in out[1:]]:
    if (status == 'Down'):
        !ansible {ugroup_name}_manager -a \
            'docker node rm {node_id}'

更新後のworkerノードの一覧を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker node ls -f role=worker'

## 管理者ユーザの設定

構築環境のJupyterHubでは講師権限を持つ管理ユーザが操作を行うためのインベントリファイルが配置されています。このインベントリを更新します。

### インベントリの更新

管理者に配布しているインベントリから削除したノードを除外します。

In [ ]:
%run scripts/cwh.py
for email in gvars.get("teacher_email_list", []):
    name = get_username_from_mail_address(email)
    !ansible-playbook -Dv -l {ugroup.name}_manager -e teacher_id={name} playbooks/deploy-inventory.yml